# Bengaluru Restaurant Prescriptive Modeling

#### Miguel Fernandez
#### BAN 530

This notebook contains work related to prescriptive modeling, often called optimization modeling. The notebook will attempt to identify which cuisines and which neighborhood a restaurant should open in to maximize its rating. Weights for the various cuisines and neighbhorhoods have been generated using values from a boosted random forest model conducted in the previous phase of the project. That work can be found here. **ADD LINK**

In [1]:
import pandas as pd
from pulp import *

In [2]:
df = pd.read_csv('sampled_data.csv')

df.head(3)

,address,name,votes,location,dish_liked,cuisines,approx_cost(for_two_people),final_rating,chain,online_order_bin,...,juices_bin,bengali_bin,bbq_bin,american_bin,healthy_food_bin,sandwich_bin,other_bin,cuisines_real_list,num_cuisines,validation
0,"16th Cross, Pragathi Layout, Doddankundi, Mara...",Palos,49,Marathahalli,NaN,"Chinese, North Indian",300.0,3.3,0,1,...,0,0,0,0,0,0,0,"['Chinese', 'NorthIndian']",2,train
1,"688, 7th Main, 10th Cross Main Road, 2nd Stage...",Roti Wala,199,BTM,"Gulab Jamun, Buttermilk, Egg Bhurji, Sprout Sa...",North Indian,300.0,4.0,0,1,...,0,0,0,0,0,0,0,['NorthIndian'],1,train
2,"117, 1st Main, Jakkasandra Extension, Koramang...",Delhi Food Point,912,Koramangala 1st Block,"Jeera Rice, Corn Paratha, Paneer Bhurji, Gobhi...","North Indian, Beverages",250.0,3.3,0,1,...,0,0,0,0,0,0,0,"['NorthIndian', 'Beverages']",2,train


In [3]:
# get neighborhood values
neighborhood_lst = df.columns[12:52].tolist()

# get cuisine values
cuisines_lst = df.columns[53:84].tolist()

In [6]:
# neighborhood_lst

In [7]:
# cuisines_lst

In [69]:
# set up model
model = LpProblem(name='restaurant_cuisine_neigborhood', sense=LpMaximize)

In [70]:
# set variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat='Binary') for i in cuisines_lst}
y = {i: LpVariable(name=f"y{i}", lowBound=0, cat='Binary') for i in neighborhood_lst}

### NEED TO ADD TONS OF CONSTRAINTS

In [71]:
# Add constraints
model += (lpSum(y.values()) == 1, 'neighborhood_selection')
model += (lpSum(x.values()) <= 4, 'cuisine_selection')

# If Indiranagar then must NOT be healthy food
model += (y['indiranagar_bin'] + x['healthy_food_bin'] <= 1, 'healthfood_not_in_indirangar')


M = 100

# If Koramangala 5th Block, then must serve Italian
model += (y['koramangala_5th_block_bin'] <= x['italian_bin'] * M, '5th_block_italian')

# If Koramangala 6th Block, then one must be Other


- If Indiranagar then must be Italian, pizza or beverage
- 
- If MG Road, then must be Italian, Asian or Thai
- 
- If Richmond Road, then must NOT be South Indian, North Indian, or Chinese
- If JP Nagar, then must NOT be South Indian, North Indian, or Chinese
- If Koramangala 7th Block, then must NOT be South Indian, North Indian, Chinese, Street Food, Fast Food
- If Shanti Nagar, then must NOT be Mughlai, North Indian, Chinese
- If Koramangala 5th Block, then must be at least one	Biryani, Seafood, Andhra, North Indian
- 

The values used in the objective function were pulled directly from the column contribution values in the JMP software for the boosted random forest model. 

In [72]:
# Set objective
model += 0.0267 * x['north_indian_bin'] + 0.0243 * x['chinese_bin'] + 0.0088 * x['south_indian_bin'] + \
         0.0108 * x['biryani_bin'] + 0.0084 * x['fast_food_bin'] + 0.0022 * x['street_food_bin'] + \
         0.0030 * x['seafood_bin'] + 0.0038 * x['continental_bin'] + 0.0036 * x['andhra_bin'] + \
         0.0030 * x['beverages_bin'] + 0.0031 * x['mughlai_bin'] + 0.0028 * x['kerala_bin'] + \
         0.0026 * x['italian_bin'] + 0.0049 * x['rolls_bin'] + 0.0025 * x['momos_bin'] + \
         0.0049 * x['arabian_bin'] + 0.0072 * x['pizza_bin'] + 0.0026 * x['burger_bin'] + \
         0.0013 * x['kebab_bin'] + 0.0013 * x['thai_bin'] + 0.0018 * x['mangalorean_bin'] + \
         0.0012 * x['asian_bin'] + 0.0017 * x['desserts_bin'] + 0.0008 * x['salad_bin'] + \
         0.0007 * x['juices_bin'] + 0.0015 * x['bengali_bin'] + 0.0023 * x['bbq_bin'] + \
         0.0007 * x['american_bin'] + 0.0020 * x['healthy_food_bin'] + 0.0003 * x['sandwich_bin'] + \
         0.0176 * x['other_bin'] + \
         0.0011 * y['btm_bin'] + 0.0018 * y['hsr_bin'] + 0.0041 * y['jayanagar_bin'] + \
         0.0039 * y['koramangala_5th_block_bin'] + 0.0005 * y['jp_nagar_bin'] + 0.0094 * y['marathahalli_bin'] + \
         0.0012 * y['whitefield_bin'] + 0.0004 * y['indiranagar_bin'] + 0.0108 * y['bannerghatta_road_bin'] + \
         0.0085 * y['bellandur_bin'] + 0.0003 * y['koramangala_6th_block_bin'] + 0.0020 * y['electronic_city_bin'] + \
         0.0006 * y['brigade_road_bin'] + 0.0014 * y['koramangala_7th_block_bin'] + \
         0.0014 * y['koramangala_1st_block_bin'] + 0.0016 * y['ulsoor_bin'] + 0.0026 * y['sarjapur_road_bin'] + \
         0.0002 * y['banashankari_bin'] + 0.0011 * y['basavanagudi_bin'] + 0.0002 * y['kalyan_nagar_bin'] + \
         0.0021 * y['richmond_road_bin'] + 0.0002 * y['frazer_town_bin'] + 0.0036 * y['brookefield_bin'] + \
         0.0008 * y['rajajinagar_bin'] + 0.0008 * y['kammanahalli_bin'] + 0.0015 * y['banaswadi_bin'] + \
         0.0007 * y['malleshwaram_bin'] + 0.0000 * y['new_bel_road_bin'] + 0.0023 * y['koramangala_4th_block_bin'] + \
         0.0018 * y['cunningham_road_bin'] + 0.0146 * y['mg_road_bin'] + 0.0025 * y['shanti_nagar_bin'] + \
         0.0006 * y['church_street_bin'] + 0.0033 * y['residency_road_bin'] + 0.0000 * y['ejipura_bin'] + \
         0.0000 * y['shivajinagar_bin'] + 0.0009 * y['domlur_bin'] + 0.0005 * y['jeevan_bhima_nagar_bin'] + \
         0.0009 * y['old_airport_road_bin'] + 0.0009 * y['st._marks_road_bin']

In [73]:
status = model.solve()

PulpSolverError: PuLP: Error while executing glpsol.exe

In [63]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [41]:
print(f"objective: {model.objective.value()}")

objective: 0.09400000000000001


In [42]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

xamerican_bin: 0
xandhra_bin: 0
xarabian_bin: 0
xasian_bin: 0
xbbq_bin: 0
xbengali_bin: 0
xbeverages_bin: 0
xbiryani_bin: 1
xburger_bin: 0
xchinese_bin: 1
xcontinental_bin: 0
xdesserts_bin: 0
xfast_food_bin: 0
xhealthy_food_bin: 0
xitalian_bin: 0
xjuices_bin: 0
xkebab_bin: 0
xkerala_bin: 0
xmangalorean_bin: 0
xmomos_bin: 0
xmughlai_bin: 0
xnorth_indian_bin: 1
xother_bin: 1
xpizza_bin: 0
xrolls_bin: 0
xsalad_bin: 0
xsandwich_bin: 0
xseafood_bin: 0
xsouth_indian_bin: 0
xstreet_food_bin: 0
xthai_bin: 0
ybanashankari_bin: 0
ybanaswadi_bin: 0
ybannerghatta_road_bin: 0
ybasavanagudi_bin: 0
ybellandur_bin: 0
ybrigade_road_bin: 0
ybrookefield_bin: 0
ybtm_bin: 0
ychurch_street_bin: 0
ycunningham_road_bin: 0
ydomlur_bin: 0
yejipura_bin: 0
yelectronic_city_bin: 0
yfrazer_town_bin: 0
yhsr_bin: 0
yindiranagar_bin: 0
yjayanagar_bin: 0
yjeevan_bhima_nagar_bin: 0
yjp_nagar_bin: 0
ykalyan_nagar_bin: 0
ykammanahalli_bin: 0
ykoramangala_1st_block_bin: 0
ykoramangala_4th_block_bin: 0
ykoramangala_5th_bloc

In [15]:
model.solver